## Deliverable 3. Create a Travel Itinerary Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

In [11]:
# 1. Read the WeatherPy_vacation.csv into a DataFrame.
vacation_df = pd.read_csv("Vacation_Search/WeatherPy_vacation.csv")
vacation_df.tail(10)

,City_ID,City,Country,Max Temp,Weather description,Lat,Lng,Hotel Name
375,686,Eston,CA,82.92,scattered clouds,51.1501,-108.7514,Hearthside Place B&B
376,687,Jiexiu,CN,82.06,overcast clouds,37.0244,111.9125,IU Hotel Jinzhong Yuhua Road
377,688,Barra Patuca,HN,79.30,broken clouds,15.8000,-84.2833,NaN
378,689,Svobodnyy,RU,77.31,overcast clouds,51.4000,128.1333,Alekseevskij Hotel
379,690,Calabozo,VE,79.25,overcast clouds,8.9242,-67.4293,Hotel Giardini
380,691,Labelle,US,84.22,overcast clouds,26.7617,-81.4384,Port LaBelle Inn
381,692,Oriximina,BR,77.14,scattered clouds,-1.7656,-55.8661,Pousada Tia Amélia
382,694,Chardara,KZ,80.13,clear sky,41.2547,67.9692,Жанаргул
383,696,Barra Do Garcas,BR,72.00,clear sky,-15.8900,-52.2567,Pousada Tropical
384,698,Lodja,CD,73.94,overcast clouds,-3.4833,23.4333,NaN


In [97]:
# Drop the rows where there is no Hotel Name.
new_vacation_df = vacation_df.dropna(axis=0, how="any", inplace=False)
new_vacation_df.head(10)

,City_ID,City,Country,Max Temp,Weather description,Lat,Lng,Hotel Name
0,0,Bud,US,71.62,overcast clouds,39.4470,-86.1758,Ted McQuinn Trust
1,5,Georgetown,MY,84.13,haze,5.4112,100.3354,Cititel Penang
2,7,Butaritari,KI,82.36,few clouds,3.0707,172.7902,Isles Sunset Lodge
3,9,Padang,ID,77.05,few clouds,-0.9492,100.3543,Mercure Padang
4,13,Palu,ID,76.03,light rain,-0.8917,119.8707,Hotel Santika Palu
5,14,Viesca,MX,91.40,few clouds,25.3500,-102.8000,LA VIEZCA (Parque Magico)
6,21,Susanville,US,89.91,clear sky,40.4163,-120.6530,Comfort Inn & Suites
7,22,Iracoubo,GF,77.34,scattered clouds,5.4802,-53.2011,Le Gîte d’Iracoubo
8,23,Praya,CV,75.74,few clouds,14.9215,-23.5087,LT Aparthotel
9,25,Matara,LK,79.97,overcast clouds,5.9485,80.5353,Amaloh Boutique Resort


In [111]:
# Refine your search with different weather criteria to get cities that are close together.
preferred_cities_df = new_vacation_df.loc[new_vacation_df['Country'] == 'JP']
preferred_cities_df.head(20)

,City_ID,City,Country,Max Temp,Weather description,Lat,Lng,Hotel Name
16,37,Shimoda,JP,84.56,overcast clouds,34.6667,138.9500,Shimoda Tokyu Hotel
31,72,Toba,JP,80.60,overcast clouds,34.4833,136.8500,ホテル アルティア鳥羽
41,94,Hasaki,JP,82.80,moderate rain,35.7333,140.8333,Inubosaki Kanko Hotel
84,187,Nishihara,JP,86.05,overcast clouds,26.1842,127.7558,Minshuku Agaihama
107,230,Iwaki,JP,89.02,overcast clouds,37.0500,140.8833,Toyoko Inn Iwaki Ekimae
120,256,Katsuura,JP,81.32,overcast clouds,35.1333,140.3000,Katsuura Hotel Mikazuki
190,379,Hirara,JP,89.46,broken clouds,24.8000,125.2833,Hotel Atoll Emerald Miyakojima
219,438,Misawa,JP,81.61,scattered clouds,40.6836,141.3597,Hoshino Resorts Aomoriya
257,507,Itoman,JP,86.77,broken clouds,26.1247,127.6694,Southern Beach Hotel & Resort Okinawa
282,541,Nanae,JP,70.03,broken clouds,41.8833,140.6833,とみや旅館


In [112]:
# 2. Using the template add the city name, the country code, the weather description and maximum temperature for the city.
info_box_content = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather description</dt><dd>{Weather description} and {Max Temp} °F</dd>
</dl>
"""

# 3a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_content.format(**row) for index, row in preferred_cities_df.iterrows()]

# 3b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = preferred_cities_df[["Lat", "Lng"]]

In [113]:
# 4a. Add a marker layer for each city to the map.
locations = preferred_cities_df[["Lat", "Lng"]]
max_temp = preferred_cities_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 4b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

In [59]:
# From the map above pick 4 cities and create a vacation itinerary route to travel between the four cities. 
# 5. Create DataFrames for each city by filtering the 'vacation_df' using the loc method. 
# Hint: The starting and ending city should be the same city.

vacation_start = vacation_df.loc[vacation_df["City"] == "Shimoda"]
vacation_end = vacation_df.loc[vacation_df["City"] == "Shimoda"]
vacation_stop1 = vacation_df.loc[vacation_df["City"] == "Katsuura"]
vacation_stop2 = vacation_df.loc[vacation_df["City"] == "Hasaki"] 
vacation_stop3 = vacation_df.loc[vacation_df["City"] == "Iwaki"] 

In [62]:
# 6. Get the latitude-longitude pairs as tuples from each city DataFrame using the to_numpy function and list indexing.
start = vacation_start[["Lat", "Lng"]].to_numpy()[0]
end = vacation_end[["Lat", "Lng"]].to_numpy()[0]
stop1 = vacation_stop1[["Lat", "Lng"]].to_numpy()[0]
stop2 = vacation_stop2[["Lat", "Lng"]].to_numpy()[0]
stop3 = vacation_stop3[["Lat", "Lng"]].to_numpy()[0]

In [63]:
# 7. Create a direction layer map using the start and end latitude-longitude pairs,
# and stop1, stop2, and stop3 as the waypoints. The travel_mode should be "DRIVING", "BICYCLING", or "WALKING".
fig = gmaps.figure()
JP_trip = gmaps.directions_layer(
        start, end, waypoints=[stop1, stop2, stop3],
        travel_mode='DRIVING')
fig.add_layer(JP_trip)
fig

Figure(layout=FigureLayout(height='420px'))

In [64]:
# 8. To create a marker layer map between the four cities.
#  Combine the four city DataFrames into one DataFrame using the concat() function.
itinerary_df = pd.concat([vacation_start, vacation_stop1, vacation_stop2, vacation_stop3],ignore_index=True)
itinerary_df

,City_ID,City,Country,Max Temp,Weather description,Lat,Lng,Hotel Name
0,37,Shimoda,JP,84.56,overcast clouds,34.6667,138.9500,Shimoda Tokyu Hotel
1,256,Katsuura,JP,81.32,overcast clouds,35.1333,140.3000,Katsuura Hotel Mikazuki
2,94,Hasaki,JP,82.80,moderate rain,35.7333,140.8333,Inubosaki Kanko Hotel
3,230,Iwaki,JP,89.02,overcast clouds,37.0500,140.8833,Toyoko Inn Iwaki Ekimae


In [75]:
# 9 Using the template add city name, the country code, the weather description and maximum temperature for the city. 
info_box_content = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather description</dt><dd>{Weather description} and {Max Temp} °F</dd>
</dl>
"""

# 10a Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_content.format(**row) for index, row in itinerary_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = itinerary_df[["Lat", "Lng"]]

In [76]:
# 11a. Add a marker layer for each city to the map.
locations = itinerary_df[["Lat", "Lng"]]
max_temp = itinerary_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))